In [ ]:
import tensorflow as tf

from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import accuracy_score, classification_report

import os
import shutil
from tqdm import tqdm
import statistics

import torch

import matplotlib.pyplot as plt

import torchvision.models as models
import torch.nn as nn
from torchvision import transforms, datasets

# Training

In [ ]:
train_data_dir = 'Train/'
valid_data_dir = 'Valid/'
test_data_dir='Test/'
# Paramètres du modèle
img_width, img_height = 512, 512
input_shape = (img_width, img_height, 3)
epochs = 20
batch_size = 32

# Prediction plusieurs lames

In [ ]:
model = torch.load('/best_model.pth')

In [ ]:
accuracy_m=[]
recall_m=[]
specificity_m=[]
report_m=[]
TEST_DIR="/Test/"
RESULT_DIR='/result_cla/'
os.makedirs(RESULT_DIR, exist_ok=True)
for lame in tqdm(os.listdir(TEST_DIR)):
    print('ok')
    test_data_dir = TEST_DIR+lame+'/'

    img_width, img_height = 512, 512
    batch_size = 32

    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',  # 'binary' pour un problème de classification binaire
        shuffle=False  # Assurez-vous que les prédictions correspondent à l'ordre des images
    )

    # Faire des prédictions sur le jeu de test
    predictions = model(test_generator)

    # Obtenir les étiquettes prédites (0 pour non tumoral, 1 pour tumoral)
    predicted_labels = (predictions > 0.5).astype(int)

    # Obtenir les étiquettes réelles
    true_labels = test_generator.classes

    # Évaluer la performance du modèle
    accuracy = accuracy_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()

    # Calculate specificity
    specificity = tn / (tn + fp)
    report = classification_report(true_labels, predicted_labels)
    accuracy_m.append(accuracy)
    recall_m.append(recall)
    specificity_m.append(specificity)
    report_m.append(report)
    print(f"Accuracy: {accuracy}")
    print("Classification Report:\n", report)

    # Dossier où enregistrer les résultats
    output_dir_tumoral = RESULT_DIR+lame+'_predict/'
    #print(test_data_dir)

    # Créer les dossiers de sortie s'ils n'existent pas
    os.makedirs(output_dir_tumoral, exist_ok=True)
    
    os.makedirs(output_dir_tumoral+'background/', exist_ok=True)
    os.makedirs(output_dir_tumoral+'lit/', exist_ok=True)


    # Itérer sur les prédictions et copier les images dans les dossiers appropriés
    for i, (filename, prediction) in enumerate(zip(test_generator.filenames, predicted_labels)):
        
        source_path = os.path.join(test_data_dir, filename)
        destination_folder = output_dir_tumoral + ('lit/' if prediction == 1 else 'background/')

        os.makedirs(destination_folder, exist_ok=True)

        destination_path = os.path.join(destination_folder)

        # Copier l'image vers le dossier approprié
        shutil.copy(source_path, destination_path)

    # Afficher la prédiction et le chemin de destination
    print(f"Image {i + 1} - Prédiction: {'lit' if prediction == 1 else 'background'}, Destination: {destination_path}")


print("Accuracy moyenne", statistics.mean(accuracy_m))
print("Sensitivity moyenne",statistics.mean(recall_m))
print("Specificity moyenne", statistics.mean(specificity_m))

# Prediction d'une lame

In [ ]:
model = torch.load('/best_model.pth')

In [ ]:
test_data_dir = '/Test/12AG01551-14_NADJ02_HES/'
os.makedirs(test_data_dir, exist_ok=True)
img_width, img_height = 512, 512
batch_size = 32

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # 'binary' pour un problème de classification binaire
    shuffle=False  # Assurez-vous que les prédictions correspondent à l'ordre des images
)

In [ ]:
# Faire des prédictions sur le jeu de test
predictions = model(test_generator)

In [ ]:


resnet = models.resnet18(pretrained=False)  # Il n'est pas nécessaire de charger les poids pré-entraînés ici

# Modification de la dernière couche pour correspondre au nombre de classes
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)  # 2 classes : lit et background

# Charger les poids du meilleur modèle entraîné
checkpoint = torch.load('/best_model.pth')

# Charger les poids du modèle pré-entraîné, sauf la couche finale (fc)
model_dict = resnet.state_dict()
pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict}
model_dict.update(pretrained_dict)
resnet.load_state_dict(model_dict)

# Passer en mode évaluation
resnet.eval()
resnet.eval()  # Passer en mode évaluation

# Définition des transformations pour les données de test
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Charger les données de test
test_data = datasets.ImageFolder(root='/Test/12AG01551-14_NADJ02_HES/', transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

# Faire des prédictions sur les données de test
predictions = []
with torch.no_grad():
    for images, _ in test_loader:
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.tolist())

# Afficher les prédictions
print("Predictions:", predictions)

In [ ]:
#Uniquement si on a les labels associer !!

# Obtenir les étiquettes prédites (0 pour non tumoral, 1 pour tumoral)
#predicted_labels = (predictions > 0.5).astype(int)

# Obtenir les étiquettes réelles
true_labels = test_generator.classes

# Évaluer la performance du modèle
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
#specificity = specificity_score(true_labels, predictions)
report = classification_report(true_labels, predictions)
print("Accuracy moyenne",accuracy)
print("Sensitivity moyenne",recall)
#print("Specificity moyenne", specificity)
print("Classification Report:\n", report)

In [ ]:
# Dossier où enregistrer les résultats
output_dir_tumoral = '/result3/12AG01551-14_NADJ02_HES/predict_lit/'
output_dir_non_tumoral = '/result3/12AG01551-14_NADJ02_HES/predict_background/'

# Créer les dossiers de sortie s'ils n'existent pas
os.makedirs(output_dir_tumoral, exist_ok=True)
os.makedirs(output_dir_non_tumoral, exist_ok=True)

# Itérer sur les prédictions et copier les images dans les dossiers appropriés
for i, filename in enumerate(test_generator.filenames):
    source_path = os.path.join(test_data_dir, filename)
    nom=filename.split('/')
    if predictions[i] == 1:
        destination_path = os.path.join(output_dir_tumoral, nom[1])
    else:
        destination_path = os.path.join(output_dir_non_tumoral, nom[1])

    # Copier l'image vers le dossier approprié
    shutil.copy(source_path, destination_path)

print("Résultats enregistrés dans les dossiers appropriés.")